In [1]:
# This cell installs the necessary packages listed in your Pluto notebook.
# You only need to run this once.
import Pkg
Pkg.add(["Plots", "ForwardDiff", "Roots", "PlutoUI", "BenchmarkTools"])

   Resolving package versions...
   Installed Accessors ────────────── v0.1.42
   Installed Hyperscript ──────────── v0.0.5
   Installed Tricks ───────────────── v0.1.12
   Installed HypertextLiteral ─────── v0.9.5
   Installed SpecialFunctions ─────── v2.6.1
   Installed PlutoUI ──────────────── v0.7.72
   Installed BenchmarkTools ───────── v1.6.2
   Installed Roots ────────────────── v2.2.10
   Installed StaticArraysCore ─────── v1.4.3
   Installed CommonSubexpressions ─── v0.3.1
   Installed Compat ───────────────── v4.18.1
   Installed CompositionsBase ─────── v0.1.2
   Installed MIMEs ────────────────── v1.1.0
   Installed DiffRules ────────────── v1.15.1
   Installed ConstructionBase ─────── v1.6.0
   Installed OpenSpecFun_jll ──────── v0.5.6+0
   Installed DiffResults ──────────── v1.1.0
   Installed InverseFunctions ─────── v0.1.17
   Installed CommonSolve ──────────── v0.2.4
   Installed IOCapture ────────────── v0.2.5
   Installed ForwardDiff ──────────── v1.2.2
   Installed 

In [2]:
using Markdown
using InteractiveUtils

In [3]:
using Plots, LinearAlgebra, ForwardDiff, Roots, PlutoUI

In [4]:
using BenchmarkTools;

# Table of Contents

In [5]:
plotly()

┌ Warning: Failed to load integration with PlotlyBase & PlotlyKaleido.
│   exception =
│    ArgumentError: Package PlotlyBase not found in current path.
│    - Run `import Pkg; Pkg.add("PlotlyBase")` to install the PlotlyBase package.
│    Stacktrace:
│      [1] macro expansion
│        @ .\loading.jl:2375 [inlined]
│      [2] macro expansion
│        @ .\lock.jl:376 [inlined]
│      [3] __require(into::Module, mod::Symbol)
│        @ Base .\loading.jl:2358
│      [4] require(into::Module, mod::Symbol)
│        @ Base .\loading.jl:2334
│      [5] top-level scope
│        @ C:\Users\tomka\.julia\packages\Plots\8ZnR3\src\backends.jl:570
│      [6] eval(m::Module, e::Any)
│        @ Core .\boot.jl:489
│      [7] _initialize_backend(pkg::Plots.PlotlyBackend)
│        @ Plots C:\Users\tomka\.julia\packages\Plots\8ZnR3\src\backends.jl:569
│      [8] backend(pkg::Plots.PlotlyBackend)
│        @ Plots C:\Users\tomka\.julia\packages\Plots\8ZnR3\src\backends.jl:245
│      [9] plotly()
│        @

Plots.PlotlyBackend()

# Code Improvement

# Tips for speeding it up

* focus on kernel routines: e.g. mvup, mvdown, findneighbors
* there are only finite number of results for findneighbors, so a look-up table for staple / Esite or dedicated fast function will be efficient
* simplicity is king!
* specify the types of inputs
* in-line, avoid nesting, check closure, type stability
* fixed / preallocated memory runs faster
  - e.g. hard code the dim = 4 case would give us faster code

* tools: @code_warntype, ProfileView, etc.

last but not least...
* DO NOT OVERDO IT

In [6]:
function old(Tnow; Ns = 8, dim = 3)

    # MC implementation of ND Ising Model

    jcoup = 1.0
    mu = 1.0
    Bnow = 0.

    beta = 1.0 / Tnow

    latt = ones(Int, ntuple(i1 -> Ns, dim))
    sites = CartesianIndices(latt)

    # handle periodicity
    function Ip(i1::Int)::Int
        ret = (i1 + Ns) % Ns
        # map 0 to Ns
        ret += (ret == 0) * Ns
        return ret
    end

    function Ipc(x::CartesianIndex)::CartesianIndex
        tmp = ntuple(i1 -> Ip(x[i1]), dim)
        return CartesianIndex(tmp)
    end

    function findneighbors(latt, site::CartesianIndex; bothway = true)
        # find all the neighbors given a site (index)

        kdelta(i, j) = (i == j) * 1
        neig = 0
        for d = 1:dim

            dx1 = ntuple(i1 -> kdelta(i1, d), dim)
            dx1 = CartesianIndex(dx1)

            n1 = Ipc(site + dx1)
            n2 = Ipc(site - dx1)

            neig += latt[n1]
            if bothway
                neig += latt[n2]
            end

        end
        return neig
    end

    
    function sweep!(latt; metro = true)

        for site in sites

            Esite = -jcoup * findneighbors(latt, site) - mu * Bnow

            Eup = Esite
            Edown = -Eup

            # if it's now up, flip -> dE = -2*Eup
            dE = latt[site] * (-2 * Eup)

            if metro

                # accept if dE < 0 or dE is not so large
                if rand() < exp(-beta * dE)
                    latt[site] *= -1
                end

            else

                # heatbath
                pplus = exp(-beta * Eup)
                pplus /= (pplus + 1 / pplus)

                if rand() < pplus
                    latt[site] = 1
                else
                    latt[site] = -1
                end
            end

        end
    end

    function update!(latt; N = 40, metro = true)
        for i1 = 1:N
            sweep!(latt, metro = metro)
        end
    end

    # measurement
    function obs(latt)

        # export total spin and total energy

        spin = 0.0
        ham = 0.0

        for site in sites

            ss = latt[site]
            spin += ss
            neig = findneighbors(latt, site, bothway = false)
            ham += (-jcoup * neig - mu * Bnow) * ss
        end
        return spin, ham
    end

    # start here:

    # stir well
    update!(latt, N = 150)

    s1 = 0.0
    s2 = 0.0
    E1 = 0.0

    Nm = 100
    for i1 = 1:Nm

        _s, _E = obs(latt)

        s1 += _s / Nm
        s2 += _s^2 / Nm

        E1 += _E / Nm

        update!(latt, N = 10)
    end

    mm = s1 / Ns^dim
    sus = (s2 - s1^2) / Ns^dim * beta
    Eav = E1 / Ns^dim

    return mm, sus, Eav

end

old (generic function with 1 method)

In [7]:
function new(Tnow, Ns=8, dim=3)

	# simplify mvup, mvdown
	# dim must be known ntuple in the same scope
	# remove garbage in findneighbors: e.g. bothway: i<j = (i != j) / 2
	# tabulate / make fast function for staple to find Esite
	
    # MC implementation of ND Ising Model
	
    jcoup = 1.0
    mu = 1.0
    Bnow = 0.01

    beta = 1.0 / Tnow

    latt = ones(Int64, ntuple(i1 -> Ns, dim))
    sites = CartesianIndices(latt)

	function mvup(x::CartesianIndex, d::Int64)

		local dim::Int64 = length(x)
		i_tmp::Int64 = x[d] == Ns ? 1 : x[d] + 1
		return CartesianIndex(ntuple(i1-> i1!=d ? x[i1] : i_tmp, dim))

    end

    function mvdown(x::CartesianIndex, d::Int64)
		
		local dim::Int64 = length(x)
		i_tmp::Int64 = x[d] == 1 ? Ns : x[d] - 1
		return CartesianIndex(ntuple(i1-> i1!=d ? x[i1] : i_tmp, dim))
		
    end
		
	function findneighbors(site::CartesianIndex)::Int64
		
        # find all the neighbors given a site (index)
		local dim = length(site)

        neig = 0
        for d = 1:dim
            neig += latt[mvup(site, d)] + latt[mvdown(site, d)]
        end
		
        return neig
		
    end

	# staple: generate a look-up table / fast function
	# staple_list = [-jcoup * nval - mu*Bnow for nval in -2*dim : 2*dim]
	staple(nval::Int64)::Float64 = -jcoup * nval - mu*Bnow

    function sweep!()

        for site in sites
			# Esite = staple_list[findneighbors(site)+2*dim+1]
			Esite = staple(findneighbors(site))

			#= 
			metropolis:
			
			dE = -2 * latt[site] * Esite
			decision to flip:
			rand() < exp(-beta * dE)
			
			=#
			
			if rand() < exp(beta * 2*latt[site]*Esite)
				latt[site] *= -1
			end
			
        end
		
    end

    function update!(; N = 40)
        for i1 = 1:N
            sweep!()
        end
    end

    # measurement
    function obs()

        # export total spin and total energy
        spin = 0.0
        ham = 0.0

        for site in sites
			lval = latt[site]
            spin += lval
            # ham += staple_list[findneighbors(site)+2*dim+1] * latt[site] / 2
			ham += staple(findneighbors(site)) * lval / 2
        end
        return spin, ham
		
    end

    # start here:

    # stir well
    update!(N = 150)

    s1 = 0.0
    s2 = 0.0
    E1 = 0.0
	E2 = 0.0

    Nm = 100
    for i1 = 1:Nm

        _s, _E = obs()

        s1 += _s / Nm
        s2 += _s^2 / Nm

        E1 += _E / Nm
		E2 += _E^2 / Nm

        update!(N = 10)
		
    end

    mm = s1 / Ns^dim
    sus = (s2 - s1^2) / Ns^dim * beta
    Eav = E1 / Ns^dim
	heatcap = (E2-E1^2) / Ns^dim * beta^2

    return mm, sus, Eav, heatcap

end

new (generic function with 3 methods)

In [8]:
let
    p1 = @benchmark $old(2.0)
    p2 = @benchmark $new(2.0)

    [p1, p2]

end

2-element Vector{BenchmarkTools.Trial}:
 1.617 s
 16.403 ms

# Code Analysis

# FASTEST ISING CODE ON EARTH

(constantly update :D)
DO NOT GO CRAZY!!

In [9]:
function pokemon1(Tnow; Ns=12, dim=3)

	# code improvement due to Biplab and Sasha!
    # MC implementation of ND Ising Model
	
    jcoup = 1.0
    mu = 1.0
    Bnow = 0.01

    beta = 1.0 / Tnow

    latt = ones(Int64, ntuple(i1 -> Ns, dim))
    sites = CartesianIndices(latt)

	function mvup(x::CartesianIndex, d::Int64)

		local dim::Int64 = length(x)
		i_tmp::Int64 = x[d] == Ns ? 1 : x[d] + 1
		return CartesianIndex(ntuple(i1-> i1!=d ? x[i1] : i_tmp, dim))

    end

    function mvdown(x::CartesianIndex, d::Int64)
		
		local dim::Int64 = length(x)
		i_tmp::Int64 = x[d] == 1 ? Ns : x[d] - 1
		return CartesianIndex(ntuple(i1-> i1!=d ? x[i1] : i_tmp, dim))
		
    end
		
	function findneighbors(site::CartesianIndex)::Int64
		
        # find all the neighbors given a site (index)
		local dim = length(site)

        neig = 0
        for d = 1:dim
            neig += latt[mvup(site, d)] + latt[mvdown(site, d)]
        end
		
        return neig
		
    end

	# staple: generate a look-up table / fast function
	# staple_list = [-jcoup * nval - mu*Bnow for nval in -2*dim : 2*dim]
	staple(nval::Int64)::Float64 = -jcoup * nval - mu*Bnow

    function sweep!()

        for site in sites
			# Esite = staple_list[findneighbors(site)+2*dim+1]
			Esite = staple(findneighbors(site))

			#= 
			metropolis:
			
			dE = -2 * latt[site] * Esite
			decision to flip:
			rand() < exp(-beta * dE)
			
			=#
			
			if rand() < exp(beta * 2*latt[site]*Esite)
				latt[site] *= -1
			end
			
        end
		
    end

    function update!(; N = 40)
        for i1 = 1:N
            sweep!()
        end
    end

    # measurement
    function obs()
        # export total spin and total energy
        spin = 0.0
        ham = 0.0
        for site in sites
			lval = latt[site]
            spin += lval
            # ham += staple_list[findneighbors(site)+2*dim+1] * latt[site] / 2
			ham += staple(findneighbors(site)) * lval / 2
        end
        return spin, ham
    end

    # start here:

    # stir well
    update!(N = 150)

    s1 = 0.0
    s2 = 0.0
    E1 = 0.0
	E2 = 0.0

    Nm = 100
    for i1 = 1:Nm

        _s, _E = obs()

        s1 += _s / Nm
        s2 += _s^2 / Nm

        E1 += _E / Nm
		E2 += _E^2 / Nm

        update!(N = 10)
		
    end

    mm = s1 / Ns^dim
    sus = (s2 - s1^2) / Ns^dim * beta
    Eav = E1 / Ns^dim
	heatcap = (E2-E1^2) / Ns^dim * beta^2

    return mm, sus, Eav, heatcap

end

pokemon1 (generic function with 1 method)

In [10]:
function pokemon2(Tnow; Ns=12, dim=3)

    # MC implementation of ND Ising Model
	# using iterators and tuples
	
    jcoup = 1.0
    mu = 1.0
    Bnow = 0.01

    beta = 1.0 / Tnow

	latt = ones(Int64, ntuple(i1 -> Ns, dim))
	# sites = Iterators.product(ntuple(i1->1:Ns, dim)...)
	sites = Tuple.(CartesianIndices(latt))
	
	function mvup(x, d::Int64)
		local dim::Int64 = length(x)
		i_tmp::Int64 = x[d] == Ns ? 1 : x[d] + 1
		return ntuple(i1-> i1!=d ? x[i1] : i_tmp, dim)
    end

    function mvdown(x, d::Int64)	
		local dim::Int64 = length(x)
		i_tmp::Int64 = x[d] == 1 ? Ns : x[d] - 1
		return ntuple(i1-> i1!=d ? x[i1] : i_tmp, dim)		
    end
		
	function findneighbors(site)::Int64
		
        # find all the neighbors given a site (index)
		local dim = length(site)

        neig = 0
        for d = 1:dim
            neig += latt[mvup(site, d)...] + latt[mvdown(site, d)...]
        end
		
        return neig
		
    end

	# staple: generate a look-up table / fast function
	# staple_list = [-jcoup * nval - mu*Bnow for nval in -2*dim : 2*dim]
	staple(nval::Int64)::Float64 = -jcoup * nval - mu*Bnow

    function sweep!()

        for site in sites
			# Esite = staple_list[findneighbors(site)+2*dim+1]
			Esite = staple(findneighbors(site))

			#= 
			metropolis:
			
			dE = -2 * latt[site] * Esite
			decision to flip:
			rand() < exp(-beta * dE)
			
			=#
			
			if rand() < exp(beta * 2*latt[site...]*Esite)
				latt[site...] *= -1
			end
			
        end
		
    end

    function update!(; N = 40)
        for i1 = 1:N
            sweep!()
        end
    end

    # measurement
    function obs()
		# export total spin and total energy
        spin = 0.0
        ham = 0.0

        for site in sites
			lval = latt[site...]
            spin += lval
            # ham += staple_list[findneighbors(site)+2*dim+1] * latt[site] / 2
			ham += staple(findneighbors(site)) * lval / 2
        end
        return spin, ham
    end

    # start here:

    # stir well
    update!(N = 150)

    s1 = 0.0
    s2 = 0.0
    E1 = 0.0
	E2 = 0.0

    Nm = 100
    for i1 = 1:Nm

        _s, _E = obs()

        s1 += _s / Nm
        s2 += _s^2 / Nm

        E1 += _E / Nm
		E2 += _E^2 / Nm

        update!(N = 10)
		
    end

    mm = s1 / Ns^dim
    sus = (s2 - s1^2) / Ns^dim * beta
    Eav = E1 / Ns^dim
	heatcap = (E2-E1^2) / Ns^dim * beta^2

    return mm, sus, Eav, heatcap

end

pokemon2 (generic function with 1 method)

In [11]:
function sasha1(T::Float64; B=0.01, Nd = 3, Ns = 12, Nstir = 150, Nm = 100, dNm = 10)
	J = 1.0
    μ = 1.0
    β = 1 / T

	# Spins
	latt = ones(Int, ntuple(i -> Ns, Nd))
	
	# Coordinate Grid
	sites = CartesianIndices(latt)
	
	# Neighbours (directional vectors)
	neigsv = [CartesianIndex(ntuple(i -> Int(x == i), Nd)) for x in 1:Nd]
	append!(neigsv, [CartesianIndex(ntuple(i -> -Int(x == i), Nd)) for x in 1:Nd])

	# Periodic Index
	PIndex(ind::CartesianIndex)::CartesianIndex = CartesianIndex(ntuple(i -> mod1(ind[i], Ns), Nd))

	# Neighbours
	neigs = [[PIndex(ind + n) for n in neigsv] for ind in sites]
	
	# Energy of one spin
	Esite(ind::CartesianIndex)::Float64 = (-J * sum(latt[n] for n in neigs[ind]) - μ * B) * latt[ind]

	# Total energy (Hamiltonian)
	Etot() = sum((-J * sum(latt[neigs[ind][n]] for n in 1:Nd) - μ * B) * latt[ind] for ind in sites)
	
	function updatesite(site)
		if rand() < exp(β * 2 * Esite(site))
			latt[site] *= -1
		end
		return nothing
	end
	
	function update(; N = 40)
		for i in 1:N
			for site in sites
				updatesite(site)
			end
		end
		return nothing
	end

    update(N = Nstir)

    s1 = 0.0
    s2 = 0.0
    Eav = 0.0

    for i1 = 1:Nm

        tmp = sum(latt)
        s1 += tmp
        s2 += tmp^2

        Eav += Etot()

        update(N = dNm)
    end

    mm = s1 / Ns^Nd / Nm
    sus = (s2 - s1^2 / Nm) / Ns^Nd * β / Nm

    return mm, sus, Eav / Ns^Nd / Nm
	
end

sasha1 (generic function with 1 method)

In [12]:
function sasha2(T::Float64, ; B = 0.01, Nd = 3, Ns = 12, Nstir = 150, Nm = 100, dNm = 10)
	J = 1.0
    μ = 1.0
    β = 1 / T

	# Spins
	latt = ones(Int, ntuple(i -> Ns, Nd))
	# Coordinate Grid
	sites = eachindex(latt)
	# Strides and increments
	str = strides(latt)
	steps = ntuple(i -> Ns * str[i], Nd)
	
	# Neighbours (with periodicity)
	nh(x::Int64, s::Int64, a::Int64)::Int64 = x + s - a * (cld(x + s, a) - cld(x, a))
	nl(x::Int64, s::Int64, a::Int64)::Int64 = x - s + a * (cld(x, a) - cld(x - s, a))

	nf = [ntuple(i -> nh(site, str[i], steps[i]), Nd) for site in sites]
	nb = [ntuple(i -> nl(site, str[i], steps[i]), Nd) for site in sites]
	#nf = [[nh(site, str[i], steps[i]) for i in 1:Nd] for site in sites]
	#nb = [[nl(site, str[i], steps[i]) for i in 1:Nd] for site in sites]
	
	sum_fneigs(site::Int64)::Int64 = sum(latt[nf[site][n]] for n in 1:Nd)	
	sum_neigs(site::Int64)::Int64 = sum(latt[nf[site][n]] + latt[nb[site][n]] for n in 1:Nd)

	# Energy of one site
	Esite(ind::Int64)::Float64 = (-J * sum_neigs(ind) - μ * B) * latt[ind]

	# Total energy (Hamiltonian)
	Etot()::Float64 = sum((-J * sum_fneigs(ind) - μ * B) * latt[ind] for ind in sites)
	
	function updatesite(site)
		if rand() < exp(β * 2 * Esite(site))
			latt[site] *= -1
		end
		return nothing
	end
	
	function update(; N = 40)
		for i in 1:N
			for site in sites
				updatesite(site)
			end
		end
		return nothing
	end
	
	# stir
    update(N = Nstir)

    s1 = 0.0
    s2 = 0.0
    Eav = 0.0

    for i1 = 1:Nm

        tmp = sum(latt)
        s1 += tmp
        s2 += tmp^2

        Eav += Etot()

        update(N = dNm)
    end

    mm = s1 / Ns^Nd / Nm
    sus = (s2 - s1^2 / Nm) / Ns^Nd * β / Nm

    return mm, sus, Eav / Ns^Nd / Nm
	
end

sasha2 (generic function with 1 method)

In [13]:
function biplab1(T;B=0.01,Ns=12,dim=3)
    ## Define Parameters
    J = 1.0
    μ = 1.0
    β = 1/T

    ## Create Lattice
    latt = ones(Int64,ntuple(i->Ns,dim))
    sites = CartesianIndices(latt)

    ## Unit Vectors
    unitvec = [CartesianIndex(ntuple(i->(x==i),dim)) for x in 1:dim]
    unitvec2 = .- unitvec

    append!(unitvec,unitvec2)

    ## Periodic Boundary Condition
    PBC(site) = CartesianIndex(mod1.(Tuple(site),Ns))

    ## List to save the neighbors data
    neighbors = [[PBC(site+eáµ¢) for eáµ¢ in unitvec] for site in sites]

    ## List of possible energy for lookup
    list_energy = [-J*neig - μ*B for neig in -2dim:2dim]

    Esite(site) = list_energy[sum(latt[neighbor] for neighbor in neighbors[site]) + 2dim + 1]

    ham()::Float64 = sum([ list_energy[sum(latt[neighbors[site][i]] for i in 1:dim) + 2dim + 1] for site in sites])

    spin()::Float64 = sum([spin for spin in latt])

    function sweep()
        for site in sites
            if rand() < exp(2β*latt[site]*Esite(site))
                latt[site] *= -1
            end
        end
    end

    function update(; N = 40, metro = true)
        for i1 = 1:N
            sweep()
        end
    end

    # stir well
    update(N = 150)

    s1 = 0.0
    s2 = 0.0
    Eav = 0.0

    Nm = 100
    for i1 = 1:Nm
        tmp = spin()
        s1 += tmp / Nm
        s2 += tmp^2 / Nm

        Eav += ham() / Ns^dim / Nm

        update(N = 10)
    end

    mm = s1 / Ns^dim
    sus = (s2 - s1^2) / Ns^dim * β

    return mm, sus, Eav
end

biplab1 (generic function with 1 method)

In [14]:
function biplab2(T::Float64;B=0.01,Ns=12,dim=3)
    ## Define Parameters
    J = 1.0
    μ = 1.0
    β = 1/T

    ## Create Lattice
    latt = ones(Int64,ntuple(i->Ns,dim))
    sites = CartesianIndices(latt)

    ## Unit Vectors
    unitvec = [CartesianIndex(ntuple(i->(x==i),dim)) for x in 1:dim]
    unitvec2 = .- unitvec

    append!(unitvec,unitvec2)

    ## Periodic Boundary Condition
    PBC(site::CartesianIndex) = CartesianIndex(mod1.(Tuple(site),Ns))

    neighbors = [[PBC(site+dx) for dx in unitvec] for site in sites]

    list_energy = [-J*neig - μ*B for neig in -2dim:2dim]

    Esite(site::CartesianIndex) = list_energy[sum(latt[neighbor] for neighbor in neighbors[site]) + 2dim + 1]

    ham()::Float64 = sum([ list_energy[sum(latt[neighbors[site][i]] for i in 1:dim)+2dim+1] for site in sites])

    spin()::Float64 = sum([spin for spin in latt])

    function sweep()
        @inbounds for site in sites
            if rand() < exp(2β*latt[site]*Esite(site))
                latt[site] *= -1
            end
        end
    end

    function update(; N = 40, metro = true)
        @inbounds for i1 = 1:N
            sweep()
        end
    end

    # stir well
    update(N = 150)

    s1 = 0.0
    s2 = 0.0
    Eav = 0.0

    Nm = 100
    @inbounds for i1 = 1:Nm
        tmp = spin()
        s1 += tmp / Nm
        s2 += tmp^2 / Nm

        Eav += ham() / Ns^dim / Nm

        update(N = 10)
    end

    mm = s1 / Ns^dim
    sus = (s2 - s1^2) / Ns^dim * β

    return mm, sus, Eav
end

biplab2 (generic function with 1 method)

In [ ]:
function kazulak0_1()
    # Lattice size:
    L = 3
    # Lattice dimensions:
    dim = 2

    # Spin grid:
    grid = ones(Int, ntuple(d -> L, dim))

    # Coupling Constant
    J = 1

    # Magnetic moment
    μ = 1 

    # External magnetic field
    h = 1

    # Temperature
    T = 2.2
    β = 1 / T

    MonteCarloSteps = 1000
    for i = 1:MonteCarloSteps
        
    end

    function neighbour_periodic_boundry_condition(index_x, index_y)
        if index_x == 1
            previous_x = L
        elseif index_x == L
            next_x = 1
        else
            next_x = index_x + 1
            previous_x = index_x - 1
        end

        if index_y == 1
            previous_y = L
        elseif index_y == L
            next_y = 1
        else
            next_y = index_y + 1
            previous_y = index_y - 1
        end
        
        #= testing:
        println(previous_x, previous_y, next_x, next_y)
        grid[previous_x, index_y] = 2
        grid[next_x, index_y] = 3
        grid[index_x, previous_y] = 4
        grid[index_x, next_y] = 5 =#

        # Sum of neighbours multiplied by Coupling Constant
        # TO DO check this
        return - J * (grid[previous_x, index_y] + grid[next_x, index_y] + grid[index_x, previous_y] + grid[index_x, next_y])
    end

    function boltzman_distribution(site_index)
        exp(-β*diff_in_energy)
    end    
    grid[2,2]=1
    # algorithms steps - TO DO
    # How to use those?
    sum = neighbour_periodic_boundry_condition(2,2)
    Hamiltonian = sum - μ * h * grid[2,2]
    println(grid, sum, Hamiltonian)
end

kazulak0_1()

[1 1 1; 1 1 1; 1 1 1]-4-5


In [18]:
let

	p1 = @benchmark sasha1(1.4)
	p2 = @benchmark sasha2(1.4)
	p3 = @benchmark biplab1(1.4)
	p4 = @benchmark biplab2(1.4)
	p5 = @benchmark pokemon1(1.4)
	p6 = @benchmark pokemon2(1.4)
	
    [p1, p2, p3, p4, p5, p6]

end

6-element Vector{BenchmarkTools.Trial}:
 94.456 ms
 65.390 ms
 65.471 ms
 65.142 ms
 56.369 ms
 66.520 ms

In [16]:
let
	Tnow = 5 * rand()
	[sasha1(Tnow), biplab1(Tnow), pokemon1(Tnow)]
end

3-element Vector{Tuple{Float64, Float64, Float64, Vararg{Float64}}}:
 (1.0, 0.0, -3.010000000000124)
 (0.9999999999999987, 2.039566226227005e-11, -3.010000000000002)
 (0.9999999999999987, 2.039566226227005e-11, -3.0050000000000856, -7.307225496206929e-11)